In [1]:
# Import Required Modules

import os
import IPython.display as ipd
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Rescaling, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, RMSprop
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings("ignore")

In [2]:
DATA_PATH = '/kaggle/input/birdfiltered/10SecDatasets/10SecDatasets'
BIRDS = os.listdir(f'{DATA_PATH}/Train')
# BIRDS = [ 'Barn_Swallow',
#     'Black-backed_Puffback',
#     'Black_Kite',
#     'Cape_Robin-Chat',
#     'Cattle_Egret',
#     'Collared_Sunbird',
#     'Common_Bulbul',
#     'Common_Buzzard',
#     'Common_House-Martin',/
#     'Common_Sandpiper',
#     'Egyptian_Goose',
#     'Eurasian_Hoopoe',
#     'European_Bee-eater',
#     'Gray-backed_Camaroptera',
#     'Great_Egret',
#     'Little_Egret',
#     'Rattling_Cisticola',
#     'Red-backed_Scrub-Robin',
#     'Red-rumped_Swallow',
#     'Sombre_Greenbul',
#     'Tawny-flanked_Prinia',
#     'Thrush_Nightingale',
#     'Western_Yellow_Wagtail',
#     'Willow_Warbler',
#     'Wood_Sandpiper']

BATCH_SIZE =8
IM_SIZE = (224, 224) 
NO_CLASSES = len(BIRDS)

In [ ]:
# Load the train, validation and testing dataset

from tensorflow.keras.applications.efficientnet import preprocess_input

# Data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode="nearest",
)
train_batches = train_datagen.flow_from_directory(
    os.path.join(DATA_PATH, "Train"),
    classes=BIRDS,
    target_size=IM_SIZE,
    class_mode="categorical",
    shuffle=True,
    batch_size=BATCH_SIZE,
)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rescale=1./255)
valid_batches = valid_datagen.flow_from_directory(
    os.path.join(DATA_PATH, "Validation"),
    classes=BIRDS,
    target_size=IM_SIZE,
    class_mode="categorical",
    shuffle=False,
    batch_size=BATCH_SIZE,
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                 rescale=1./255)
test_batches = test_datagen.flow_from_directory(
    os.path.join(DATA_PATH, "Test"),
    classes=BIRDS,
    target_size=IM_SIZE,
    class_mode="categorical",
    shuffle=False,
    batch_size=BATCH_SIZE,
)

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0

lr = 0.0001
loss = "categorical_crossentropy"

# Transfer learning EfficientNetB3
model = EfficientNetB0(
    include_top=False, weights="imagenet", input_tensor=None, input_shape=(IM_SIZE[0],IM_SIZE[1], 3)
)
x = model.output
x = Flatten()(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
output_layer = Dense(len(BIRDS), activation="softmax", name="softmax")(x)
model = Model(inputs=model.input, outputs=output_layer)
model.compile(
    optimizer=Adam(learning_rate=lr), loss=loss, metrics=["accuracy"]
)

# print(model.summary())

In [ ]:
# Define callbacks

import datetime
# Model checkpoint
logs_directory = 'model/logs/'
checkpoint = ModelCheckpoint(
    r"model/model_checkpoint.keras",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    mode="auto",
    save_freq='epoch',
)

# ReduceLR = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=3e-4)
# Function for exponential learning rate decay after 5 epochs
def scheduler(epoch, lr):
  decay = -0.1
  if epoch < 5:
    return lr
  else:
    return float(lr * math.exp((epoch+1)*decay))

lrscheduler = LearningRateScheduler(scheduler)

# logdir = os.path.join("logsMFCC", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard = TensorBoard(log_dir=logdir,histogram_freq=1)

# Early stopping if val_loss does not decrease for 3 epochs
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=True)

callbacks = [checkpoint, lrscheduler, early_stopping_monitor]
# callbacks = [tensorboard]

In [ ]:
# EPOCHS = 30
# steps_per_epoch = train_batches.samples // BATCH_SIZE
# validation_steps = valid_batches.samples // BATCH_SIZE

# history = model.fit(
#     train_batches,
#     validation_data=valid_batches,
#     epochs=EPOCHS,
#     callbacks=callbacks,
#     steps_per_epoch=steps_per_epoch,
#     validation_steps=validation_steps,
#     verbose=1  # Change to 2 for more compact output
# )

# # Optionally save the training history
# import json
# with open('training_history.json', 'w') as f:
#     json.dump(history.history, f)


In [ ]:
# Train the model

EPOCHS = 30
history = model.fit(
    train_batches,
    validation_data=valid_batches,
    epochs=EPOCHS,
    callbacks= callbacks)

In [ ]:
model.save('./model/EFF_v5.keras')
model_loaded = load_model('./model/EFF_v5.keras')
model_loaded.evaluate(test_batches)

In [ ]:
model.evaluate(test_batches)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# Confusion Matrix

def confusion_matrix_plot(model, test_batches):
    
    annot_size = 24
    fig_size = (20, 20)

    preds = model_loaded.predict(x=test_batches)
    y_preds = np.argmax(preds, axis=-1)
    y_true = test_batches.classes

    confuse = confusion_matrix(y_true, y_preds)

    plt.rcParams["font.family"] = "Times New Roman"
    sns.set(rc = {'figure.figsize':fig_size, 'xtick.labelsize':annot_size, 'ytick.labelsize':annot_size})
    # plt.rcParams['figure.figsize'] = fig_size
    conf_df = pd.DataFrame(confuse)
    conf_df.index = BIRDS
    conf_df.columns = BIRDS
    sns.heatmap(conf_df, annot=True, cmap='binary', annot_kws={'size': annot_size})
    plt.title('Confusion Matrix', fontdict={'fontsize':annot_size}) 
    plt.xlabel('Predicted Birds', fontdict={'fontsize':annot_size})
    plt.ylabel('Actual Birds', fontdict={'fontsize':annot_size})
    plt.savefig('Confusion Matrix.png', dpi=200, format='png', bbox_inches='tight')

confusion_matrix_plot(model, test_batches)

In [ ]:
# Classes with scores heatmap
from sklearn.metrics import classification_report
def class_scores(classes_dir, model, test_batches):
    
    birds_train_count = {}
    annot_size = 14
    fig_size = (8,20)
    
    for f in os.listdir(classes_dir):
        birds_train_count[f] = len(os.listdir(os.path.join(train_dir, f)))

    birds_train_count = dict(sorted(birds_train_count.items(), key= lambda x: x[1], reverse=True))

    preds = model.predict(x=test_batches)
    y_preds = np.argmax(preds, axis=-1)
    y_true = test_batches.classes
    clf_report = classification_report(y_true, y_preds, output_dict=True)
    cls_df = pd.DataFrame(clf_report).iloc[:-1, :].T
    acc_to_counts = [str(BIRDS.index(bird)) for bird in [k for k in birds_train_count.keys()]] + ['accuracy', 'macro avg', 'weighted avg']

    cls_df = cls_df.reindex(acc_to_counts)
    y_label = [f"{k} ({v})" for k, v in birds_train_count.items()] + ['accuracy', 'macro avg', 'weighted avg']
    cls_df.index = y_label

    sns.set(rc = {'figure.figsize':fig_size})
    sns.heatmap(cls_df, annot=True, annot_kws={'size': annot_size}, cmap='binary')
    plt.title('Classes with Scores Heatmap') 
    plt.savefig('Classes with Scores Heatmap.png', dpi=200, format='png', bbox_inches='tight')

train_dir = '/kaggle/input/birdfiltered/10SecDatasets/10SecDatasets/Train'

class_scores(train_dir, model_loaded, test_batches)

In [ ]:
import shutil

dir_name = '/kaggle/working/model/EFF_v2.keras'
output_filename = dir_name.split('/')[-1]

shutil.make_archive(output_filename, 'zip', dir_name)

In [ ]:
# Accuracy and Loss 

from sklearn.metrics import classification_report, f1_score, confusion_matrix
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
sns.set()
sns.set_style(style='white')

# Accuracy vs epochs plot
def accuracy_plot(model, history, epochs):

    annot_size = 10
    fig_size = (20, 20)

    plt.plot(history.history['accuracy'], label='Training Accuracy', color='black')
    plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy', linestyle='dashed', color='black')
    
    plt.title('Model Accuracy', fontsize=annot_size)
    plt.xlabel('Epoch', fontsize=annot_size)
    plt.ylabel('Accuracy', fontsize=annot_size)
    plt.ylim([0, 1])
    plt.xticks(list(range(1, epochs+1)))
    plt.legend(loc='lower right', fontsize=annot_size)
    plt.savefig('accuracy2.jpg', bbox_inches='tight')
    plt.show()

# Loss vs epochs plot
def loss_plot(model, history, epochs):
    annot_size = 10
    fig_size = (20, 20)

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.plot(loss, label='Training Loss', color='black')
    plt.plot(val_loss, label = 'Validation Loss', linestyle='dashed', color='black')
    
    plt.title('Model Loss', fontsize=annot_size)
    plt.xlabel('Epochs', fontsize=annot_size)
    plt.ylabel('Loss', fontsize=annot_size)
#     plt.ylim([0, 0.85])
    plt.xticks(list(range(1, epochs+1)))
    plt.legend(loc='upper right', fontsize=annot_size)
    plt.savefig('loss2.jpg', bbox_inches='tight')
    plt.show()

accuracy_plot(model, history, epochs=15)
loss_plot(model, history, epochs=15)

In [ ]:
# Inference

# !pip install pydub
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Rescaling

from pydub import AudioSegment
import math
import shutil

import warnings
warnings.filterwarnings("ignore")

# # Turn interactive plotting off
# import matplotlib
# matplotlib.use('Agg')
# plt.ioff()

# Variables for feature extraction
SAMPLE_RATE = 32000
SPEC_SHAPE = (48, 128) # height x width
SIGNAL_LENGTH = 10
N_FFT = 1024         
HOP_SIZE =  int(SIGNAL_LENGTH * SAMPLE_RATE / (SPEC_SHAPE[1] - 1))      
N_MELS = SPEC_SHAPE[0]   
WIN_SIZE = 1024      
WINDOW_TYPE = 'hann' 
FEATURE = 'mel'      
FMIN = 500
FMAX = 12500

def SplitAudio(audio_file, sec_to_split=10):
    
    validAudio = lambda audio, amp_threshold: True if audio.max > amp_threshold else False
    
    global temp_n
    temp_n = 0
    mili=1000
    export_folder = f'./inference/tmp'
    export_name = audio_file.split('/')[-1]
    exportFormat = 'mp3'
    try:
        audio = AudioSegment.from_mp3(audio_file)
    except:
        print("File handled: ", audio_file)
        extension = audio_file.split('.')[-1]
        if extension == 'wav':
            audio = AudioSegment.from_wav(audio_file)
        elif extension == 'ogg':
            audio = AudioSegment.from_ogg(audio_file)
    th = audio.max//2
 
    duration = math.floor(audio.duration_seconds)
    splits = np.arange(0, duration, sec_to_split)

    if duration > sec_to_split:
        rem = duration % sec_to_split
        n_splits = round(duration / sec_to_split)
        for n in range(n_splits-1): # Except last splitted file
            newAudio = audio[splits[n]*mili:splits[n+1]*mili]
            temp_n = n
            if validAudio(newAudio, th):
                newAudio.export(f'{export_folder}/{export_name}{n}.{exportFormat}', format=exportFormat)
        
        # For the last splitted file
        if rem <= sec_to_split//2:
            n = temp_n
            # If remaining  audio is small enough, don't separate
            newAudio = audio[splits[n+1]*mili:duration*mili]
            if validAudio(newAudio, th):
                newAudio.export(f'{export_folder}/{export_name}{n+1}.{exportFormat}', format=exportFormat)

        else:
            n = temp_n
            # If remaining audio is big enough, make it separate file
            newAudio = audio[splits[n+1]*mili:duration*mili]
            if validAudio(newAudio, th):
                newAudio.export(f'{export_folder}/{export_name}{n+1}.{exportFormat}', format=exportFormat)

    else:
        if validAudio(audio, th):
            audio.export(f'{export_folder}/{export_name}original.{exportFormat}', format=exportFormat)

def create_features(inference_data_path):
    
    # Find the list of all 10 sec splitted audio files
    audio_files = [f for f in os.listdir(inference_data_path) if f.split('.')[-1]=='mp3']
    count = 0
    for aud in audio_files:

        signal, sr = librosa.load(os.path.join(inference_data_path, aud),duration=10) # sr = sampling rate
        # Plot mel-spectrogram
        S = librosa.feature.melspectrogram(y=signal,
                                           sr=SAMPLE_RATE,
                                            n_fft=N_FFT,
                                            hop_length=HOP_SIZE, 
                                            n_mels=N_MELS, 
                                            fmin=FMIN, 
                                            fmax=FMAX) 

        fig = plt.figure(figsize=(10, 4))
        mel_spec = librosa.power_to_db(S, ref=np.max) 
        librosa.display.specshow(mel_spec, fmin=FMIN,y_axis='linear')
        
        plt.axis(False)
        plt.tight_layout()
        # plt.show()
        count += 1
        plt.savefig(f'{inference_data_path}/inf{count}.jpg')
        plt.close(fig)
        
def preprocess_img(image_dir, img_size):
    img_width, img_height = img_size[0], img_size[1]
    img = image.load_img(image_dir, target_size = (img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    norm = Rescaling(1./255)
    img = norm(img)
    img = preprocess_input(img) 
    return img

def prediction(model_dir, inference_data_path, img_size):
    # List of feature images
    feat = [os.path.join(inference_data_path, f) for f in os.listdir(inference_data_path) if f.split('.')[-1]=='jpg']
    results = []
    for m in feat:
        model_loaded = load_model(model_dir)
        img = preprocess_img(m, img_size)
        output = BIRDS[np.argmax(model_loaded.predict(img), axis=-1)[0]]
        results.append(output)
    result = max(results)
    
    # Remove the temporary splitted audio and mfcc images
    for i in os.listdir(inference_data_path):
        os.remove(os.path.join(inference_data_path, i))
    return results

In [ ]:
inference_data_path = './inference/tmp'
raw_data_path = '/kaggle/input/birdfiltered/XC700260 - Great Hornbill - Buceros bicornis.mp3'
model_dir = '/kaggle/working/model/CNMelSpec_Model'
img_size = (128, 128)

if 'inference' not in os.listdir('./'):
    os.makedirs(inference_data_path)

SplitAudio(raw_data_path)
create_features(inference_data_path)
result = prediction(model_dir, inference_data_path, img_size)
print(max(result))